# Bloodhound and Sharphound Hack Tool
Detects command line parameters used by Bloodhound and Sharphound hack tools

## Rule Content
```
- title: Bloodhound and Sharphound Hack Tool
  id: f376c8a7-a2d0-4ddc-aa0c-16c17236d962
  description: Detects command line parameters used by Bloodhound and Sharphound hack
    tools
  author: Florian Roth
  references:
  - https://github.com/BloodHoundAD/BloodHound
  - https://github.com/BloodHoundAD/SharpHound
  date: 2019/12/20
  modified: 2019/12/21
  tags:
  - attack.discovery
  - attack.t1087
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image|contains:
      - \Bloodhound.exe
      - \SharpHound.exe
    selection2:
      CommandLine|contains:
      - ' -CollectionMethod All '
      - '.exe -c All -d '
      - Invoke-Bloodhound
      - Get-BloodHoundData
    selection3:
      CommandLine|contains|all:
      - ' -JsonFolder '
      - ' -ZipFileName '
    selection4:
      CommandLine|contains|all:
      - ' DCOnly '
      - ' --NoSaveCache '
    condition: 1 of them
  falsepositives:
  - Other programs that use these command line option and accepts an 'All' parameter
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\Bloodhound.exe* OR *\\SharpHound.exe*) OR process_command_line.keyword:(*\ \-CollectionMethod\ All\ * OR *.exe\ \-c\ All\ \-d\ * OR *Invoke\-Bloodhound* OR *Get\-BloodHoundData*) OR (process_command_line.keyword:*\ \-JsonFolder\ * AND process_command_line.keyword:*\ \-ZipFileName\ *) OR (process_command_line.keyword:*\ DCOnly\ * AND process_command_line.keyword:*\ \-\-NoSaveCache\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()